# Timothy Sims
## A Python notebooklassification of guns by their sound using machine learning

# PLoading data and running LSTM model

In [5]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf

train_data = pd.read_csv('train.csv')   # load data train and val
val_data = pd.read_csv('val.csv')
test_data = pd.read_csv('test.csv')

train_data['cleaned_text'] = train_data['abstract_text'].str.lower()    # clean text
val_data['cleaned_text'] = val_data['abstract_text'].str.lower()
test_data['cleaned_text'] = test_data['abstract_text'].str.lower()

vocab_size = 60000 
max_length = 64
trunc_type = 'post'
padding_type = 'post'
oov_tok = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)  # tokenize
tokenizer.fit_on_texts(train_data['cleaned_text'])

def get_sequences(tokenizer, data):
    sequences = tokenizer.texts_to_sequences(data)
    padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
    return padded

train_sequences = get_sequences(tokenizer, train_data['cleaned_text'])  # convert to sequences and pad
val_sequences = get_sequences(tokenizer, val_data['cleaned_text'])
test_sequences = get_sequences(tokenizer, test_data['cleaned_text'])

train_labels = pd.get_dummies(train_data['target']).values  # convert to one-hot
val_labels = pd.get_dummies(val_data['target']).values
test_labels = pd.get_dummies(test_data['target']).values

model = tf.keras.Sequential([   # build model
    tf.keras.layers.Embedding(vocab_size, 100, input_length=max_length),    # embedding layer
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50)),    # bidirectional LSTM layer
    tf.keras.layers.Dropout(0.5),   # dropout layer
    tf.keras.layers.Dense(50, activation='relu'),   # dense layer
    tf.keras.layers.Dropout(0.5),   # dropout layer
    tf.keras.layers.Dense(len(train_data['target'].unique()), activation='softmax') # output layer
])


model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])  # compile model
model.summary() # print model summary

history = model.fit(train_sequences, train_labels, epochs=5, validation_data=(val_sequences, val_labels))   # train model

test_loss, test_acc = model.evaluate(test_sequences, test_labels)   # evaluate model
print(f"Test Accuracy: {test_acc}") # print test accuracy


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 64, 100)           6000000   
                                                                 
 bidirectional_4 (Bidirecti  (None, 100)               60400     
 onal)                                                           
                                                                 
 dropout_2 (Dropout)         (None, 100)               0         
                                                                 
 dense_7 (Dense)             (None, 50)                5050      
                                                                 
 dropout_3 (Dropout)         (None, 50)                0         
                                                                 
 dense_8 (Dense)             (None, 5)                 255       
                                                      

In [ ]:
# I changed the model to include two dropout layers and a dense layer with a size of 50
# I also changed the embedding size to 100 and the LSTM layer to 50
# Doing these along with changing the epoch to 5 allowed me to improve the accuracy by 2%

# Combining LSTM and CNN models

# LSTM+CNN model

In [ ]:
import os
import numpy as np
import librosa
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Bidirectional, LSTM, Reshape
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report


def read_data(folder_path, fft_size=1024, hop_length=None, win_length=None):    # added fft size hop length and win length as extra parameters
    labels = []
    spectrograms = []

    for label in os.listdir(folder_path):
        subfolder_path = os.path.join(folder_path, label)
        if os.path.isdir(subfolder_path):
            for file in os.listdir(subfolder_path):
                file_path = os.path.join(subfolder_path, file)
                if file_path.endswith('.wav'):
                    y, sr = librosa.load(file_path)
                    spectrogram = librosa.stft(y, n_fft=fft_size, hop_length=hop_length, win_length=win_length)
                    spectrogram = np.abs(spectrogram)
                    spectrograms.append(spectrogram)
                    labels.append(label)

    return spectrograms, labels

def pad2d(a, desired_size):
    rows, cols = a.shape
    padded_a = np.zeros((desired_size, desired_size))
    rows_to_copy = min(rows, desired_size)
    cols_to_copy = min(cols, desired_size)
    padded_a[:rows_to_copy, :cols_to_copy] = a[:rows_to_copy, :cols_to_copy]
    return padded_a

def create_lstm_cnn_model(input_shape, num_classes):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))   # conv2d 32 3x3
    model.add(MaxPooling2D((2, 2))) # maxpool 2x2
    model.add(BatchNormalization()) # batchnorm
    model.add(Conv2D(64, (4, 4), activation='relu'))    # changed to 4x4
    model.add(MaxPooling2D((2, 2))) # maxpool 2x2
    model.add(BatchNormalization()) # batchnorm
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(BatchNormalization())
    model.add(Conv2D(256, (5, 5), activation='relu'))   # newly added conv2d 256 5x5
    model.add(MaxPooling2D((2, 2))) # maxpool 2x2
    model.add(BatchNormalization()) # batchnorm
    model.add(Flatten())    # flatten the model

    model.add(Reshape((80, 80)))    # reshape to 80x80 for lstm
    
    model.add(Bidirectional(LSTM(64, return_sequences=False)))  # bidirectional lstm 64

    model.add(Dense(128, activation='relu'))    # dense 128 relu
    model.add(Dropout(0.5)) # dropout 0.5

    model.add(Dense(num_classes, activation='softmax')) # dense for output
    return model

folder_path = 'Gun Shot dataset/edge-collected-gunshot-audio'

spectrograms, labels = read_data(folder_path, fft_size=2048, hop_length=512, win_length=2048)   # added fft size hop length and win length as extra parameters


desired_spectrogram_size = 128  
spectrograms = np.array([pad2d(s, desired_spectrogram_size) for s in spectrograms])
spectrograms = np.expand_dims(spectrograms, axis=-1) 
print(spectrograms.shape)
label_dict = {label: i for i, label in enumerate(set(labels))}
y = np.array([label_dict[label] for label in labels])
y = to_categorical(y) 

X_train, X_test, y_train, y_test = train_test_split(spectrograms, y, test_size=0.2, random_state=42)

input_shape = X_train[0].shape
num_classes = y.shape[1]

model = create_lstm_cnn_model(input_shape, num_classes)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test))

lstm_cnn_test_loss, lstm_cnn_test_accuracy = model.evaluate(X_test, y_test)  # evaluate model
print(f"Test Accuracy: {lstm_cnn_test_accuracy}")   # print test accuracy
print(f"Test loss: {lstm_cnn_test_loss}")   # print test loss

y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

lstm_cnn_report = classification_report(y_true_classes, y_pred_classes, target_names=label_dict.keys())  # classification report to show precision, recall, and f1 score

print(lstm_cnn_report)


(2148, 128, 128, 1)
Epoch 1/100
54/54 [==============================] - 10s 137ms/step - loss: 1.2963 - accuracy: 0.3964 - val_loss: 1.3238 - val_accuracy: 0.3093
Epoch 2/100
54/54 [==============================] - 7s 131ms/step - loss: 1.0978 - accuracy: 0.5233 - val_loss: 1.2405 - val_accuracy: 0.4233
Epoch 3/100
54/54 [==============================] - 7s 134ms/step - loss: 0.9289 - accuracy: 0.6170 - val_loss: 1.2492 - val_accuracy: 0.4977
Epoch 4/100
54/54 [==============================] - 8s 140ms/step - loss: 0.6623 - accuracy: 0.7456 - val_loss: 1.0850 - val_accuracy: 0.5488
Epoch 5/100
54/54 [==============================] - 7s 138ms/step - loss: 0.4523 - accuracy: 0.8283 - val_loss: 1.0989 - val_accuracy: 0.5814
Epoch 6/100
54/54 [==============================] - 7s 133ms/step - loss: 0.3364 - accuracy: 0.8818 - val_loss: 0.8141 - val_accuracy: 0.6860
Epoch 7/100
54/54 [==============================] - 7s 129ms/step - loss: 0.1741 - accuracy: 0.9424 - val_loss: 0.7071 -

In [51]:
# implementing the LSTM part of the model was a bit challenging as I had to reshape the data to fit the LSTM layer
# eventually using the output given from errors I was able to reshape the data and get the model to work
# I had tried more LSTM layers like I had in the LSTM only model but had to remove them as they only hurt the accuracy of the model
# I also tried different numbers of epochs before settling on 100, I tried lower numbers however the accuracy was not as good
# thought the val accuracy drops often after some epochs, in the end it recovers and gives a good final accuracy

# the accuracy in the end was .97 or 97% which is quite good
# the model is most precise with the Remington 870 at 99% and least precise with the Glock 17 at 94%
# this means that when the model predicts the Glock, it is 3% more likely to be wrong than when it predicts the Remington 870

# the category with the highest recall is a tie between the 38 S&W and the Glock 17 with 99%, the lowest recall is the Remington 870 with 91%
# this means that when the model predicts that it is NOT a Remington 870, it is 8% more likely to be wrong than when it predicts that it is NOT a 38 S&W or a Glock 17

# The highest F1 score is the 38 S&W with 99%, the lowest is the Remington 870 with 95%
# this means that overall the model is 4% more accurate when predicting the 38 S&W than when predicting the Remington 870
# The performance of the model on the Remington 870 is the worst overall and in recall, but this is most likely because the Remington 870 has the fewest samples in the dataset

# LSTM only model

In [ ]:
import os
import numpy as np
import librosa
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Bidirectional, LSTM, Reshape
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report


def read_data(folder_path, fft_size=1024, hop_length=None, win_length=None):
    labels = []
    spectrograms = []

    for label in os.listdir(folder_path):
        subfolder_path = os.path.join(folder_path, label)
        if os.path.isdir(subfolder_path):
            for file in os.listdir(subfolder_path):
                file_path = os.path.join(subfolder_path, file)
                if file_path.endswith('.wav'):
                    y, sr = librosa.load(file_path)
                    spectrogram = librosa.stft(y, n_fft=fft_size, hop_length=hop_length, win_length=win_length)
                    spectrogram = np.abs(spectrogram)
                    spectrograms.append(spectrogram)
                    labels.append(label)

    return spectrograms, labels

def pad2d(a, desired_size):
    rows, cols = a.shape
    padded_a = np.zeros((desired_size, desired_size))
    rows_to_copy = min(rows, desired_size)
    cols_to_copy = min(cols, desired_size)
    padded_a[:rows_to_copy, :cols_to_copy] = a[:rows_to_copy, :cols_to_copy]
    return padded_a

def create_lstm_model(input_shape, num_classes):
    model = Sequential()

    model.add(Bidirectional(LSTM(64, return_sequences=True), input_shape=input_shape))  # bidirectional lstm 64
    model.add(Dropout(0.5)) # dropout 0.5

    model.add(Bidirectional(LSTM(64, return_sequences=False)))  # bidirectional lstm 64
    model.add(Dropout(0.5)) # dropout 0.5

    model.add(Dense(128, activation='relu'))    # dense 128
    model.add(Dropout(0.5)) # dropout 0.5

    model.add(Dense(num_classes, activation='softmax')) # dense for output

    return model

folder_path = 'Gun Shot dataset/edge-collected-gunshot-audio'

spectrograms, labels = read_data(folder_path, fft_size=2048, hop_length=512, win_length=2048)


desired_spectrogram_size = 128  
spectrograms = np.array([pad2d(s, desired_spectrogram_size) for s in spectrograms]) # pad
spectrograms = np.expand_dims(spectrograms, axis=-1)    # expand dims
spectrograms = spectrograms.reshape(spectrograms.shape[0], desired_spectrogram_size, -1)    # reshape
print(spectrograms.shape)   # print shape
label_dict = {label: i for i, label in enumerate(set(labels))}  # create label dict
y = np.array([label_dict[label] for label in labels])
y = to_categorical(y)  

X_train, X_test, y_train, y_test = train_test_split(spectrograms, y, test_size=0.2, random_state=42)    # train test split

input_shape = X_train[0].shape
num_classes = y.shape[1]

model = create_lstm_model(input_shape, num_classes) # create model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])  # compile model

model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test))   # train model

y_pred = model.predict(X_test)  # predict
y_pred_classes = np.argmax(y_pred, axis=1)  # get classes
y_true_classes = np.argmax(y_test, axis=1)

lstm_test_loss, lstm_test_accuracy = model.evaluate(X_test, y_test) # evaluate model
print(f"Test Accuracy: {lstm_test_accuracy}")   # print test accuracy
print(f"Test loss: {lstm_test_loss}")   # print test loss

y_pred = model.predict(X_test)  # predict
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

lstm_report = classification_report(y_true_classes, y_pred_classes, target_names=label_dict.keys()) # classification report to show precision, recall, and f1 score

print(lstm_report)  # print report

(2148, 128, 128)
Epoch 1/100
54/54 [==============================] - 8s 83ms/step - loss: 1.3456 - accuracy: 0.3475 - val_loss: 1.2021 - val_accuracy: 0.4721
Epoch 2/100
54/54 [==============================] - 4s 70ms/step - loss: 1.1351 - accuracy: 0.5052 - val_loss: 1.0573 - val_accuracy: 0.5465
Epoch 3/100
54/54 [==============================] - 4s 72ms/step - loss: 0.9764 - accuracy: 0.5908 - val_loss: 0.8719 - val_accuracy: 0.6233
Epoch 4/100
54/54 [==============================] - 4s 73ms/step - loss: 0.8109 - accuracy: 0.6874 - val_loss: 1.0386 - val_accuracy: 0.6256
Epoch 5/100
54/54 [==============================] - 4s 71ms/step - loss: 0.6945 - accuracy: 0.7311 - val_loss: 0.7732 - val_accuracy: 0.6884
Epoch 6/100
54/54 [==============================] - 4s 70ms/step - loss: 0.6488 - accuracy: 0.7520 - val_loss: 0.7369 - val_accuracy: 0.7047
Epoch 7/100
54/54 [==============================] - 4s 71ms/step - loss: 0.5663 - accuracy: 0.7905 - val_loss: 0.5368 - val_accura

In [54]:
# The LSTM model was slightly more accurate than the provided CNN model, with an accuracy of 92% compared to 90%
# however it was difficult to improve it further, and ultimately needed to be combined with the CNN model to get the best results
# In the individual LSTM model I had more layers of LSTM and dropout compared to the combined model
# these were removed from the combined model as they were not helping, after removing them the accuracy increased to 97% where it stayed

# The LSTM model alone is not sufficient for this task as CNN or the combined model

# CNN only model

In [ ]:
import os
import numpy as np
import librosa
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical



def read_data(folder_path, fft_size=1024, hop_length=None, win_length=None):
    labels = []
    spectrograms = []

    for label in os.listdir(folder_path):
        subfolder_path = os.path.join(folder_path, label)
        if os.path.isdir(subfolder_path):
            for file in os.listdir(subfolder_path):
                file_path = os.path.join(subfolder_path, file)
                if file_path.endswith('.wav'):
                    y, sr = librosa.load(file_path)
                    spectrogram = librosa.stft(y, n_fft=fft_size, hop_length=hop_length, win_length=win_length)
                    spectrogram = np.abs(spectrogram)
                    spectrograms.append(spectrogram)
                    labels.append(label)

    return spectrograms, labels


def pad2d(a, desired_size):
    rows, cols = a.shape
    padded_a = np.zeros((desired_size, desired_size))
    rows_to_copy = min(rows, desired_size)
    cols_to_copy = min(cols, desired_size)
    padded_a[:rows_to_copy, :cols_to_copy] = a[:rows_to_copy, :cols_to_copy]
    return padded_a

def create_cnn_model(input_shape, num_classes):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))   # conv2d 32 3x3
    model.add(MaxPooling2D((2, 2))) # maxpool 2x2
    model.add(BatchNormalization()) # batchnorm
    model.add(Conv2D(64, (4, 4), activation='relu'))    # changed to 4x4
    model.add(MaxPooling2D((2, 2))) # maxpool 2x2
    model.add(BatchNormalization()) # batchnorm
    model.add(Conv2D(128, (3, 3), activation='relu'))   # conv2d 128 3x3
    model.add(MaxPooling2D((2, 2))) # maxpool 2x2
    model.add(BatchNormalization()) # batchnorm
    model.add(Conv2D(256, (5, 5), activation='relu'))   # newly added conv2d 256 5x5
    model.add(MaxPooling2D((2, 2))) # maxpool 2x2
    model.add(BatchNormalization()) # batchnorm
    model.add(Flatten())    # flatten the model
    
    model.add(Dense(128, activation='relu'))    # dense 128
    model.add(Dropout(0.5)) # dropout 0.5
    model.add(Dense(num_classes, activation='softmax')) # dense for output
    return model

folder_path = 'Gun Shot dataset/edge-collected-gunshot-audio'

spectrograms, labels = read_data(folder_path, fft_size=2048, hop_length=512, win_length=2048)   # added fft size hop length and win length as extra parameters



desired_spectrogram_size = 128
spectrograms = np.array([pad2d(s, desired_spectrogram_size) for s in spectrograms]) # pad
spectrograms = np.expand_dims(spectrograms, axis=-1)    # expand dims
print(spectrograms.shape)   # print shape
label_dict = {label: i for i, label in enumerate(set(labels))}
y = np.array([label_dict[label] for label in labels])
y = to_categorical(y)  

X_train, X_test, y_train, y_test = train_test_split(spectrograms, y, test_size=0.2, random_state=42)    # train test split

input_shape = X_train[0].shape
num_classes = y.shape[1]

model = create_cnn_model(input_shape, num_classes)  # create model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])  # compile model

model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test))   # train model

cnn_test_loss, cnn_test_accuracy = model.evaluate(X_test, y_test)   # evaluate model
print(f"Test Accuracy: {cnn_test_accuracy}")    # print test accuracy
print(f"Test loss: {cnn_test_loss}")    # print test loss

y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

cnn_report = classification_report(y_true_classes, y_pred_classes, target_names=label_dict.keys())  # classification report to show precision, recall, and f1 score

print(cnn_report)   # print report

(2148, 128, 128, 1)
Epoch 1/100
54/54 [==============================] - 7s 107ms/step - loss: 2.4161 - accuracy: 0.3696 - val_loss: 2.8922 - val_accuracy: 0.3860
Epoch 2/100
54/54 [==============================] - 6s 105ms/step - loss: 1.4986 - accuracy: 0.4529 - val_loss: 1.6074 - val_accuracy: 0.3512
Epoch 3/100
54/54 [==============================] - 6s 107ms/step - loss: 1.3690 - accuracy: 0.4534 - val_loss: 1.2929 - val_accuracy: 0.4372
Epoch 4/100
54/54 [==============================] - 6s 112ms/step - loss: 1.2685 - accuracy: 0.5169 - val_loss: 1.2785 - val_accuracy: 0.4907
Epoch 5/100
54/54 [==============================] - 6s 106ms/step - loss: 1.1705 - accuracy: 0.5215 - val_loss: 1.1517 - val_accuracy: 0.5070
Epoch 6/100
54/54 [==============================] - 6s 103ms/step - loss: 1.1247 - accuracy: 0.5512 - val_loss: 1.0685 - val_accuracy: 0.5814
Epoch 7/100
54/54 [==============================] - 6s 105ms/step - loss: 0.9496 - accuracy: 0.5821 - val_loss: 1.0961 - 

In [53]:
# For this model I added an extra 256 5x5 layer and changed the size of the filters in the second layer to 4x4
# this helped improve the model overall
# I also attempted to add multiple dense layers, dropout layers, and more conv layers
# The added complexity did not help however and only hurt the accuracy, so I trimmed it down to what is seen above

# I tried cutting down on epochs to avoid overfitting and help on runtime, but this caused the accuracy to drop
# this means that there is not much overfitting in the model

# Conclusion

In [49]:
print("LSTM CNN")
print(lstm_cnn_report)
print("LSTM")
print(lstm_report)
print("CNN")
print(cnn_report)

LSTM CNN
                             precision    recall  f1-score   support

       38s&ws_dot38_caliber       0.98      0.99      0.99       102
ruger_ar_556_dot223_caliber       0.98      0.97      0.97       120
     remington_870_12_gauge       0.99      0.91      0.95        78
       glock_17_9mm_caliber       0.94      0.99      0.97       130

                   accuracy                           0.97       430
                  macro avg       0.97      0.96      0.97       430
               weighted avg       0.97      0.97      0.97       430

LSTM
                             precision    recall  f1-score   support

       38s&ws_dot38_caliber       0.88      0.98      0.93       102
ruger_ar_556_dot223_caliber       0.94      0.93      0.93       120
     remington_870_12_gauge       0.93      0.86      0.89        78
       glock_17_9mm_caliber       0.92      0.90      0.91       130

                   accuracy                           0.92       430
               

In [56]:
# Overall of the three models I implemented between LSTM, CNN, and CNN LSTM, the CNN+LSTM model performed the best
# it had the highest accuracy, f1 score, and recall with the exception of the Glock 17 recall which was better in the CNN model
# as well as the 38 S&W precision which was better in the CNN model

# The CNN+LSTM model performed the best overall most likely due to the fact that it combines the two models
# this allows it to take advantage of the strengths of both models and minimize the weaknesses
# the CNN model is good at extracting features from the data, while the LSTM model is good at processing sequences
# this means that the CNN+LSTM model is good at extracting features from the data and processing them in sequence
# it had a overall accuracy of 97%, higher than the CNN model by 2% and the LSTM model by 5%

# the LSTM model performed the worst overall, this is most likely due to the fact that it is not as good at extracting features from the data
# this is because it is not meant for image data, but rather for sequence data

# I am surpised that the CNN model performed better than the combined model in the case of the Glock 17 recall and the 38 S&W precision
# this may just be the result of the final run of the model and may not be consistent
# though I am not surprised that the CNN model performed better than the LSTM model overall
# For the Glock 17 recall, I am very surprised that it reached 1.00, this may be a one off event
# but this means that out of the times that the model predicted that it was NOT a Glock 17, it was never wrong 

# The F1 score for the Remington 870 was the lowest in all three models, most likely due to the fact that it has the fewest samples in the dataset
# this means that the model has less data to learn from and is less accurate when predicting the Remington 870
# However the 38 S&W had the highest F1 score in all three models, however it did not have the most samples
# this may be due to it producting the most distinct sound out of these guns, and thus being easier to identify

# In conclusion the best model for this task is the CNN+LSTM model, as it combines the strengths of both models
# It was able to achieve the highest average in accuracy, precision, recall, and F1 score